In [59]:
import pandas as pd
import numpy as np

In [81]:
df_jan = pd.read_parquet("./data/yellow_tripdata_2022-01.parquet")
df_feb = pd.read_parquet("./data/yellow_tripdata_2022-02.parquet")

In [82]:
df_jan.shape[1]

19

In [83]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60
df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds() / 60
df_jan['duration'].std()

46.44530513776802

In [98]:
df_jan_cleaned = df_jan.query("1 <= duration <= 60 ")
df_feb_cleaned = df_feb.query("1 <= duration <= 60 ")

In [85]:
df_jan_cleaned.shape[0]/df_jan.shape[0]

0.9827547930522406

In [86]:
from sklearn.feature_extraction import DictVectorizer

In [87]:
df_jan_cleaned.shape[0]

2421440

In [94]:
v =DictVectorizer()
X = v.fit_transform(df_jan_cleaned[['PULocationID', 'DOLocationID']].astype(str).to_dict('records'))

In [89]:
X

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

In [90]:
from sklearn.linear_model import LinearRegression

In [91]:
lm = LinearRegression()
y_true = df_jan_cleaned['duration']
lm.fit(X=X, y=y_true)

LinearRegression()

In [92]:
y_predict = lm.predict(X)


rmse = np.sqrt(((y_predict-y_true)**2).mean())
rmse

6.986190814952337

In [101]:
X_val = v.transform(df_feb_cleaned[['PULocationID', 'DOLocationID']].astype(str).to_dict('records'))
y_val = df_feb_cleaned['duration']

y_val_predict = lm.predict(X_val)
rmse = np.sqrt(((y_val_predict-y_val)**2).mean())
rmse

7.786407163179794